In [53]:
import os
import requests
from bs4 import BeautifulSoup
import polars as pl

In [3]:
stranka = requests.get("https://en.wikipedia.org/wiki/List_of_canonised_popes")

In [9]:
soup = BeautifulSoup(stranka.text)

In [11]:
tabulky = soup.find_all("table")

In [39]:
for t in tabulky:
    if "Pope Clement I" in t.text:
        svati = []
        for radek in t.find_all('tr'):
            try:
                svati.append("https://en.wikipedia.org" + radek.find_all('td')[0].find_all('a')[0]['href'])
            except:
                pass
    if "Pope Urban V" in t.text:
        blahoslaveni = []
        for radek in t.find_all('tr'):
            try:
                blahoslaveni.append("https://en.wikipedia.org" + radek.find_all('td')[0].find_all('a')[0]['href'])
            except:
                pass

In [41]:
len(svati)

81

In [43]:
len(blahoslaveni)

10

In [59]:
df = pl.read_parquet(
    'data_raw/papezstvo_raw.parquet', use_pyarrow=True
).select(
    ['id','jmeno','jmeno_en','wiki_en']
).with_columns(
    pl.when(
        pl.col("wiki_en").is_in(svati)
    ).then(
        pl.lit("svatý")
    ).when(
        pl.col("wiki_en").is_in(blahoslaveni)
    ).then(
        pl.lit('blahoslavený')
    ).otherwise(
        None
    ).alias('status')
)

In [61]:
df

id,jmeno,jmeno_en,wiki_en,status
str,str,str,str,str
"""Q33923""","""svatý Petr""","""Saint Peter""","""https://en.wikipedia.org/wiki/…","""svatý"""
"""Q47144""","""Linus""","""Linus""","""https://en.wikipedia.org/wiki/…","""svatý"""
"""Q80450""","""Anaklét""","""Anacletus""","""https://en.wikipedia.org/wiki/…","""svatý"""
"""Q42887""","""Klement I.""","""Clement I""","""https://en.wikipedia.org/wiki/…",null
"""Q47746""","""Evaristus""","""Evaristus""","""https://en.wikipedia.org/wiki/…","""svatý"""
…,…,…,…,…
"""Q16975""","""Pavel VI.""","""Paul VI""","""https://en.wikipedia.org/wiki/…","""svatý"""
"""Q37278""","""Jan Pavel I.""","""John Paul I""","""https://en.wikipedia.org/wiki/…","""blahoslavený"""
"""Q989""","""Jan Pavel II.""","""John Paul II""","""https://en.wikipedia.org/wiki/…","""svatý"""


In [63]:
df.write_parquet(os.path.join('data','svati_blahoslaveni.parquet'), use_pyarrow=True)
df.write_json(os.path.join('data','svati_blahoslaveni.json'))